In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('DDosData/DDos_multi.csv')

In [ ]:
LDAP = pd.read_csv('DDosData/LDAP_clean.csv')

In [ ]:
MSSQL = pd.read_csv('DDosData/MSSQL_clean.csv')
NetBIOS = pd.read_csv('DDosData/NetBIOS_clean.csv')
Portmap = pd.read_csv('DDosData/Portmap_clean.csv')
Syn = pd.read_csv('DDosData/Syn_clean.csv')
UDP = pd.read_csv('DDosData/UDP_clean.csv')
BENIGN = pd.read_csv('DDosData/BENIGN_ddos_clean.csv')

In [ ]:
ddos = pd.concat([LDAP, MSSQL, NetBIOS, Portmap, Syn, UDP, BENIGN], ignore_index = 1)

In [ ]:
LDAP = None; del LDAP
MSSQL = None; del MSSQL
NetBIOS = None; del NetBIOS
Portmap = None; del Portmap
Syn = None; del Syn
UDP = None; del UDP
BENIGN = None; del BENIGN

In [ ]:
ddos.to_csv('DDos_all.csv')

In [ ]:
ddos.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount

In [ ]:
ddos = ddos.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
ddos.shape

(19639360, 78)

In [ ]:
ddos['Label'].value_counts()

MSSQL      5787453
Syn        4284751
UDP        3754680
NetBIOS    3657497
LDAP       1915122
Portmap     186960
BENIGN       52897
Name: Label, dtype: int64

In [ ]:
df = ddos.copy()
cat_columns = df.select_dtypes(['object']).columns
cat_columns

Index(['Label'], dtype='object')

In [ ]:
df[cat_columns] = df[cat_columns].astype('category')
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df
ddos = df
ddos.dtypes

FlowDuration               float64
TotalFwdPackets            float64
TotalBackwardPackets       float64
TotalLengthofFwdPackets    float64
TotalLengthofBwdPackets    float64
                            ...   
IdleMean                   float64
IdleStd                    float64
IdleMax                    float64
IdleMin                    float64
Label                         int8
Length: 78, dtype: object

In [ ]:
X = ddos.iloc[:, 0:-1].values
y = ddos.iloc[:, -1].values

In [ ]:
ddos = None; del ddos

In [ ]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X= standardScaler.fit_transform(X)
X

array([[-0.12880758,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.15248456, -0.00550824,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.13130602,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       ...,
       [-0.1524877 , -0.01230297,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.1524872 , -0.01230297,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141],
       [-0.14469833,  0.00808122,  0.21245269, ..., -0.12429176,
        -0.14826039, -0.13491141]])

In [ ]:
from keras.utils import np_utils
X=np.expand_dims(X,axis=2)
X.shape

(19639360, 77, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
X_train.shape

(13158371, 77, 1)

In [ ]:
X_test.shape


(6480989, 77, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(77,1)))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
import time

start = time.time()
print("Timer Begin")

model.fit(X_train,Y_train,batch_size=30,epochs=10,
          validation_split=0.1, callbacks=[EarlyStopping(monitor='loss',patience=2)])

end = time.time()
print(end - start)

Timer Begin
Epoch 1/10
394752/394752 [==============================] - 1803s 5ms/step - loss: 0.1120 - accuracy: 0.9717 - val_loss: 0.1062 - val_accuracy: 0.9729
Epoch 2/10
394752/394752 [==============================] - 1835s 5ms/step - loss: 0.1107 - accuracy: 0.9731 - val_loss: 0.1048 - val_accuracy: 0.9728
Epoch 3/10
394752/394752 [==============================] - 1803s 5ms/step - loss: 0.1190 - accuracy: 0.9730 - val_loss: 0.1019 - val_accuracy: 0.9756
Epoch 4/10
394752/394752 [==============================] - 1816s 5ms/step - loss: 0.1140 - accuracy: 0.9726 - val_loss: 0.1017 - val_accuracy: 0.9750
7296.319401979446


In [ ]:
model.save('DDos_all.h5')

In [ ]:
_, accuracy = model.evaluate(X_test, Y_test, batch_size=30, verbose=0)
print(accuracy)

0.9753766059875488


In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
rf_conf_mat = confusion_matrix(Y_test, y_pred)
print("Transfer learning confusion matrix: \n", rf_conf_mat)

Transfer learning confusion matrix: 
 [[  17054       5       0      21       0     239       1]
 [    119  621932   10684       2       0      49       3]
 [    170   41018 1852841       8       0      32   15325]
 [    375       0     270 1204820       0      21    1927]
 [     82       0     259   61013       0      69      98]
 [    908       0    1485      22       0 1411108     214]
 [     16       0   25038     103       0       8 1213650]]
